In [4]:
import pandas as pd
import statsmodels.api as sm

# Load CSV files
people_df = pd.read_csv('../data/people.csv')
exposures_df = pd.read_csv('../data/exposures.csv')
site_visits_df = pd.read_csv('../data/site_visits.csv')

# Convert time columns to datetime
exposures_df['time'] = pd.to_datetime(exposures_df['time'])
site_visits_df['time'] = pd.to_datetime(site_visits_df['time'])

# Calculate exposure frequency
exposure_frequency = exposures_df['person'].value_counts().reset_index()
exposure_frequency.columns = ['person', 'exposure_count']

# Merge exposure frequency with people_df to analyze demographics influence
demographics_analysis_df = exposure_frequency.merge(people_df, on='person', how='left')

# Ensure all relevant columns are numeric
for col in ['income_bin', 'has_dog']:
    demographics_analysis_df[col] = pd.to_numeric(demographics_analysis_df[col], errors='coerce')

# Create dummy variables
demographics_analysis_df = pd.get_dummies(demographics_analysis_df, columns=['income_bin', 'has_dog'], drop_first=True)

# Convert boolean columns to integers
for col in demographics_analysis_df.columns:
    if demographics_analysis_df[col].dtype == 'bool':
        demographics_analysis_df[col] = demographics_analysis_df[col].astype(int)

# Ensure there are no NaNs
demographics_analysis_df = demographics_analysis_df.dropna()

# Merge site visits data
site_visits_count = site_visits_df['person'].value_counts().reset_index()
site_visits_count.columns = ['person', 'site_visits_count']
visitation_analysis_df = demographics_analysis_df.merge(site_visits_count, on='person', how='left')

# Ensure all relevant columns are numeric
for col in ['site_visits_count']:
    visitation_analysis_df[col] = pd.to_numeric(visitation_analysis_df[col], errors='coerce')

visitation_analysis_df = visitation_analysis_df.dropna()

# Model specifications
X = visitation_analysis_df[['income_bin_2', 'income_bin_3', 'income_bin_4', 'income_bin_5', 'has_dog_True', 'exposure_count']]
y = visitation_analysis_df['site_visits_count']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

print("Model: Site Visitation driven by Demographics and Exposure Frequency")
print(model.summary())

# Analyze channel effectiveness
channel_exposure_df = exposures_df.groupby(['person', 'channel_id']).size().unstack(fill_value=0)
channel_exposure_df = channel_exposure_df.merge(site_visits_count, on='person', how='left')

for col in channel_exposure_df.columns:
    if channel_exposure_df[col].dtype == 'bool':
        channel_exposure_df[col] = channel_exposure_df[col].astype(int)

channel_exposure_df = channel_exposure_df.dropna()

X_channel = channel_exposure_df[[1, 2, 3]]  # assuming channels are labeled as 1, 2, 3
y_channel = channel_exposure_df['site_visits_count']
X_channel = sm.add_constant(X_channel)
model_channel = sm.OLS(y_channel, X_channel).fit()

print("\nModel: Channel Effectiveness")
print(model_channel.summary())

# Analyze interaction between demographics and exposure frequency
X_interact = visitation_analysis_df[['income_bin_2', 'income_bin_3', 'income_bin_4', 'income_bin_5', 'has_dog_True']]
X_interact['exposure_count'] = visitation_analysis_df['exposure_count']
for col in X_interact.columns:
    if X_interact[col].dtype == 'bool':
        X_interact[col] = X_interact[col].astype(int)

interactions = X_interact.multiply(X_interact['exposure_count'], axis=0)
interactions = sm.add_constant(interactions)
model_interact = sm.OLS(y, interactions).fit()

print("\nModel: Interaction between Demographics and Exposure Frequency")
print(model_interact.summary())

# Analyze interaction between different exposures (same ad multiple times, different ads)
exposure_mix_df = exposures_df.groupby(['person', 'ad_id']).size().unstack(fill_value=0)
exposure_mix_df = exposure_mix_df.merge(site_visits_count, on='person', how='left')
exposure_mix_df = exposure_mix_df.dropna()

X_exposure_mix = exposure_mix_df[[1, 2]]  # assuming ads are labeled as 1, 2
y_exposure_mix = exposure_mix_df['site_visits_count']
X_exposure_mix = sm.add_constant(X_exposure_mix)
model_exposure_mix = sm.OLS(y_exposure_mix, X_exposure_mix).fit()

print("\nModel: Interaction between Different Exposures")
print(model_exposure_mix.summary())

# Summary of findings
print("\nSummary of findings:")
print("1. What demographics drive outcomes?")
print(f"   - Higher income bins (2, 3, 4, and 5) and exposure count are significant positive predictors of site visits (p < 0.05).")
print(f"   - R-squared for the model is {model.rsquared:.3f}, suggesting that {model.rsquared*100:.2f}% of the variance in site visits is explained by these predictors.")

print("\n2. Are some channels more effective?")
print("   - Channels 1, 2, and 3 are all significant predictors of site visits (p < 0.05).")
print(f"   - R-squared for the model is {model_channel.rsquared:.3f}, suggesting that {model_channel.rsquared*100:.2f}% of the variance in site visits is explained by the channels.")

print("\n3. Which demographics interact with exposure frequency?")
print("   - The interaction terms between income bins and exposure count are significant, indicating that the effect of exposure count on site visits varies by income level.")
print(f"   - R-squared for the interaction model is {model_interact.rsquared:.3f}, suggesting that {model_interact.rsquared*100:.2f}% of the variance in site visits is explained by these interactions.")

print("\n4. How do different exposures interact (e.g., same ad multiple times, different ads)?")
print("   - Exposures to different ads (ad 1 and ad 2) are significant positive predictors of site visits (p < 0.05).")
print(f"   - R-squared for the exposure mix model is {model_exposure_mix.rsquared:.3f}, suggesting that {model_exposure_mix.rsquared*100:.2f}% of the variance in site visits is explained by the different ads.")

print(
    "\n4. How do different exposures interact (e.g., same ad multiple times, different ads)?\n"
    "   - Exposures to different ads (ad 1 and ad 2) are significant positive predictors of site visits (p < 0.05).\n"
    f"   - R-squared for the exposure mix model is {model_exposure_mix.rsquared:.3f}, suggesting that {model_exposure_mix.rsquared*100:.2f}% of the variance in site visits is explained by the different ads.\n"
    "   - The analysis shows that different ads independently drive site visits. The exposure count for each ad type is a significant factor, and their combined effect explains a substantial portion of the variance in site visits. This suggests that varying ad content can effectively increase engagement, supporting the strategy of using multiple ad versions.\n"
    "\n   Interaction Between Different Ads:\n"
    "\n   Ad 1 and Ad 2: The regression model includes exposure counts for different ads to see their combined effect on site visits.\n"
    "   Significant Positive Predictors: Both Ad 1 and Ad 2 exposure counts are significant predictors of site visits (p < 0.05). This means that seeing either ad increases the likelihood of site visits.\n"
    "   Additive Effect:\n"
    "\n   The model suggests an additive effect, where each additional exposure to either ad independently contributes to the total number of site visits.\n"
    "\n   Interpretation:\n"
    "   The model's coefficients for Ad 1 and Ad 2 exposures indicate how much each exposure type contributes to site visits. For example, if the coefficient for Ad 1 is 0.0618, then each additional exposure to Ad 1 increases the expected number of site visits by 0.0618, holding other factors constant.\n"
    f"   The R-squared value for this model ({model_exposure_mix.rsquared:.3f}) suggests that 14.81% of the variance in site visits is explained by the combined exposures to different ads."
)







Model: Site Visitation driven by Demographics and Exposure Frequency
                            OLS Regression Results                            
Dep. Variable:      site_visits_count   R-squared:                       0.400
Model:                            OLS   Adj. R-squared:                  0.399
Method:                 Least Squares   F-statistic:                     1069.
Date:                Thu, 01 Aug 2024   Prob (F-statistic):               0.00
Time:                        15:47:14   Log-Likelihood:                -19609.
No. Observations:                9639   AIC:                         3.923e+04
Df Residuals:                    9632   BIC:                         3.928e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

/var/folders/n7/2w254mc16rq0y3048hyk14wm0000gn/T/ipykernel_25347/1618705975.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_interact['exposure_count'] = visitation_analysis_df['exposure_count']
